In [28]:
from pymongo import MongoClient
# Connect to a local MongoDB instance
client = MongoClient('mongodb://localhost:27017/')
db = client['mflix']
theaters_collection = db.get_collection('theaters')

In [11]:
theaters_collection.find_one()

{'_id': ObjectId('59a47286cfa9a3a73e51e72c'),
 'theaterId': 1000,
 'location': {'address': {'street1': '340 W Market',
   'city': 'Bloomington',
   'state': 'MN',
   'zipcode': '55425'},
  'geo': {'type': 'Point', 'coordinates': [-93.24565, 44.85466]}}}

1: Top N cities with the maximum number of theatres

In [13]:
def top_10_cities_with_max_theatres():
    # Pipeline to find top 10 cities with the maximum number of theaters
    pipeline = [
        {"$group": {
            "_id": "$location.address.city",
            "theater_count": {"$sum": 1}
        }},
        {"$sort": {"theater_count": -1}},
        {"$limit": 10}
    ]

    # Execute the aggregation pipeline
    top_cities = list(theaters_collection.aggregate(pipeline))

    # Print the results
    print("Top 10 cities with the maximum number of theaters:")
    for city_data in top_cities:
        print(f"{city_data['_id']}: {city_data['theater_count']} theaters")


top_10_cities_with_max_theatres()


Top 10 cities with the maximum number of theaters:
Las Vegas: 29 theaters
Houston: 22 theaters
San Antonio: 14 theaters
Orlando: 13 theaters
Los Angeles: 12 theaters
Dallas: 12 theaters
Atlanta: 10 theaters
San Francisco: 9 theaters
Jacksonville: 9 theaters
Minneapolis: 8 theaters


2: Top N theatres nearby given coordinates

In [43]:
theaters_collection.create_index([("location.geo", "2dsphere")])


'location.geo_2dsphere'

In [68]:
from pymongo import MongoClient

def top_n_theatres_nearby(coordinates, n):

    # Find theatres near given coordinates
    query = {
        "location.geo": {
            "$nearSphere": {
                "$geometry": {
                    "type": "Point",
                    "coordinates": coordinates
                },
                "$maxDistance": 5000
            }
        }
    }
    
    # Limit the result to top N theatres
    result = theaters_collection.find(query).limit(n)
    
    # Return the result
    return list(result)

# Example usage:
coordinates = [-93.24565, 44.85466]  # Example coordinates
n = 10  # Number of theatres to retrieve
top_theatres = top_n_theatres_nearby(coordinates, n)
for theatre in top_theatres:
    print("Theater with theaterId:",theatre['theaterId'],"city: ",theatre['location']['address']['city'])


Theater with theaterId: 1000 city:  Bloomington
Theater with theaterId: 8006 city:  Minneapolis
Theater with theaterId: 8064 city:  Minneapolis
Theater with theaterId: 8106 city:  Minneapolis
Theater with theaterId: 8127 city:  Minneapolis
Theater with theaterId: 8108 city:  Minneapolis
Theater with theaterId: 8005 city:  Minneapolis
Theater with theaterId: 281 city:  Richfield
Theater with theaterId: 2387 city:  Richfield
Theater with theaterId: 8553 city:  Richfield
